In [20]:
from nltk.corpus import stopwords
import pandas as pd
import re
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [21]:
train = pd.read_csv('EI-oc-En-sadness-train.txt', sep='\t', error_bad_lines=False)
train.columns = ["ID", "Tweet", "Affect_Dimension", "Intensity_Class"]
dev = pd.read_csv('EI-oc-En-sadness-dev.txt', sep='\t', error_bad_lines=False)
dev.columns = ["ID", "Tweet", "Affect_Dimension", "Intensity_Class"]
test= pd.read_csv("EI-oc-En-sadness-test.txt", sep='\t', error_bad_lines=False)
test.columns = ["ID", "Tweet", "Affect_Dimension", "Intensity_Class"]

<ipython-input-21-7d573cc8508f>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train = pd.read_csv('EI-oc-En-sadness-train.txt', sep='\t', error_bad_lines=False)
<ipython-input-21-7d573cc8508f>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dev = pd.read_csv('EI-oc-En-sadness-dev.txt', sep='\t', error_bad_lines=False)
<ipython-input-21-7d573cc8508f>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test= pd.read_csv("EI-oc-En-sadness-test.txt", sep='\t', error_bad_lines=False)


In [22]:
test

,ID,Tweet,Affect_Dimension,Intensity_Class
0,2018-En-03239,@TerraJole is a bully. plain and simple.,sadness,NONE
1,2018-En-02839,@LondonEconomic Sometimes our judiciary just l...,sadness,NONE
2,2018-En-01035,#pause You gotta luv March Madness. Mad?? More...,sadness,NONE
3,2018-En-01920,Massive night tonight with the decider! Who yo...,sadness,NONE
4,2018-En-03155,@RGUpdate Have you tried English hospital food...,sadness,NONE
...,...,...,...,...
970,2018-En-01005,@Thischarminham @BrentSpiner @jonathansfrakes ...,sadness,NONE
971,2018-En-01962,My neck still hurting though... 😥,sadness,NONE
972,2018-En-03336,A pineapple is neither an apple or a pine. It ...,sadness,NONE
973,2018-En-02228,@Number10gov @dwppressoffice #heaviness; That ...,sadness,NONE


In [23]:
train['Intensity_Class'] = pd.Categorical(train['Intensity_Class'])
train['Intensity_Class'] =train.Intensity_Class.cat.codes

dev['Intensity_Class'] = pd.Categorical(dev['Intensity_Class'])
dev['Intensity_Class'] = dev.Intensity_Class.cat.codes

test['Intensity_Class'] = pd.Categorical(test['Intensity_Class'])
test['Intensity_Class'] = test.Intensity_Class.cat.codes

In [24]:
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stop_words.update(["now","let", 'zero', 'one', 'two', 'three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)

#Removing Stop Words

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

train['Tweet'] = train['Tweet'].apply(removeStopWords)
test['Tweet'] = test['Tweet'].apply(removeStopWords)
dev['Tweet'] = dev['Tweet'].apply(removeStopWords)

#Stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

train['Tweet'] = train['Tweet'].apply(stemming)
dev['Tweet'] = dev['Tweet'].apply(stemming)
test['Tweet'] = test['Tweet'].apply(stemming)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
from keras.utils import to_categorical
target=train.Intensity_Class.values
y=to_categorical(target)

target=dev.Intensity_Class.values
c=to_categorical(target)

max_features = 13000
max_words = 50
batch_size = 128
epochs = 3
num_classes=4

X_train = train["Tweet"]
X_val = dev["Tweet"]
Y_train = y
Y_val = c

In [26]:
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,GRU,LSTM,Embedding
from keras.optimizers import Adam
from keras.layers import SpatialDropout1D,Dropout,Bidirectional,Conv1D,GlobalMaxPooling1D,MaxPooling1D,Flatten
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [27]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
from keras.preprocessing.text import Tokenizer

In [29]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

In [30]:
X_test = tokenizer.texts_to_sequences(test['Tweet'])
X_test =pad_sequences(X_test, maxlen=max_words)

In [31]:
X_train =pad_sequences(X_train, maxlen=max_words)
X_val = pad_sequences(X_val, maxlen=max_words)
X_test =pad_sequences(X_test, maxlen=max_words)

In [32]:
model3_LSTM=Sequential()
model3_LSTM.add(Embedding(max_features,100,mask_zero=True))
model3_LSTM.add(LSTM(64,dropout=0.4,return_sequences=True))
model3_LSTM.add(LSTM(32,dropout=0.5,return_sequences=False))
model3_LSTM.add(Dense(num_classes,activation='sigmoid'))
model3_LSTM.compile(loss='binary_crossentropy',optimizer=Adam(lr = 0.001),metrics=['accuracy'])
model3_LSTM.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 100)         1300000   
                                                                 
 lstm_2 (LSTM)               (None, None, 64)          42240     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dense_1 (Dense)             (None, 4)                 132       
                                                                 
Total params: 1,354,788
Trainable params: 1,354,788
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [33]:
history3=model3_LSTM.fit(X_train, Y_train, validation_data=(X_val, Y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/3
12/12 [==============================] - 16s 553ms/step - loss: 0.6824 - accuracy: 0.3549 - val_loss: 0.6689 - val_accuracy: 0.4257
Epoch 2/3
12/12 [==============================] - 3s 235ms/step - loss: 0.6200 - accuracy: 0.3875 - val_loss: 0.5975 - val_accuracy: 0.4307
Epoch 3/3
12/12 [==============================] - 3s 234ms/step - loss: 0.5621 - accuracy: 0.3875 - val_loss: 0.5756 - val_accuracy: 0.4332


In [34]:
#y_pred3=model3_LSTM.predict_classes(X_test, verbose=1)
import numpy as np
predict_x=model3_LSTM.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

classes_x

31/31 [==============================] - 5s 31ms/step


array([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,